In [1]:
from google.colab import drive
drive.mount('/MultiClusterGAN')
%cd /MultiClusterGAN/MyDrive/CNS/
%pip install  umap-learn scanpy leidenalg

Mounted at /MultiClusterGAN
/MultiClusterGAN/MyDrive/CNS
     |████████████████████████████████| 86 kB 5.3 MB/s 
     |████████████████████████████████| 2.0 MB 24.5 MB/s 
     |████████████████████████████████| 1.4 MB 75.9 MB/s 
     |████████████████████████████████| 1.1 MB 78.6 MB/s 
     |████████████████████████████████| 91 kB 11.2 MB/s 
     |████████████████████████████████| 3.1 MB 84.7 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.2-py3-none-any.whl size=82708 sha256=2cb47345e47d6a8efaf6534653cd8a56404852cf500655a9e9e47eba9c0b242c
  Stored in directory: /root/.cache/pip/wheels/84/1b/c6/aaf68a748122632967cef4dffef68224eb16798b6793257d82
  Created wheel for pynndescent: filename=pynndescent-0.5.6-py3-none-any.whl size=53943 sha256=82973e45f595da77d2464875737554004720e3fee04b34e5eebabd88434348e0
  Stored in directory: /root/.cache/pip/wheels/03/f1/56/f80d72741e400345b5a5b50ec3d929aca581bf45e0225d5c50
  Created 

In [2]:
# Loading the Packages

%reload_ext autoreload
%autoreload 2

import os
import warnings 
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns

from tifffile import tifffile
from scipy import ndimage

# from starmap.obj import STARMapDataset, load_data
# import starmap.analyze as anz
# import starmap.viz as viz
# import starmap.sc_util as su
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import plotly.graph_objects as go
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
from anndata import AnnData
from sklearn.cluster import AgglomerativeClustering


In [3]:
tissue_region=sc.read_h5ad('data/20220310_tissue_region_revised.h5ad')

In [6]:
tissue_region.obs

,sample,col,row,tissue,anatomical_region_label,anatomical_parent_region_label,anatomical_region_id,revised_tissue
0,sagittal3,32050.916667,376.777778,45,root,root,NaN,45+Merge_7
1,sagittal3,31180.109375,730.031250,45,root,root,NaN,45+Merge_7
2,sagittal3,31231.711111,995.088889,45,root,root,NaN,45+Merge_7
3,sagittal3,31235.095238,808.761905,45,root,root,NaN,45+Merge_7
4,sagittal3,31441.180723,854.253012,45,root,root,NaN,45+Merge_7
...,...,...,...,...,...,...,...,...
47602-7,well3_5,12529.654135,30102.127820,34,Striatum,Cerebral nuclei,NaN,34
47603-7,well3_5,12392.578512,30191.504132,6,Olfactory tubercle,Striatum ventral region,NaN,Layer1+hippo-merge_0_0_3_7
47604-7,well3_5,12836.442308,29772.788462,34,lateral olfactory tract body,lateral olfactory tract general,NaN,34
47605-7,well3_5,13028.637931,29868.379310,34,lateral olfactory tract body,lateral olfactory tract general,NaN,34


In [4]:
adata_celltype=sc.read_h5ad('adata_celltype_tissuereshape.h5ad')

In [7]:
adata_celltype.obs=adata_celltype.obs.drop(['tissue'], axis=1)

In [8]:
adata_celltype.obs

,orginindex,col,row,z,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,n_genes,...,Rank1_Defined,Rank2_Defined,Rank3_Defined,Rank4_Defined,Rank1_Refine,Rank2_Refine,Rank3_Refine,Rank4_Refine,Rank5_Symbol,Rank5_Leiden_label
977515,11.0,32050.916667,376.777778,1.888889,sagittal3,25,3.258097,36.0,3.610918,25.0,...,Neurons,CNS neurons,Spinal cord neurons,NaN,Neurons,CNS neurons,Hindbrain neurons/Spinal cord neurons,Hindbrain neurons/Spinal cord neurons,INH_10,8-0
977516,13.0,31180.109375,730.031250,4.734375,sagittal3,49,3.912023,64.0,4.174387,49.0,...,Neurons,CNS neurons,Spinal cord neurons,NaN,Neurons,CNS neurons,Hindbrain neurons/Spinal cord neurons,Hindbrain neurons/Spinal cord neurons,INH_10,8-0
977517,14.0,31231.711111,995.088889,4.088889,sagittal3,29,3.401197,45.0,3.828641,29.0,...,Immune cells,Immune cells,Immune cells,Perivascular macrophages,Immune cells,Immune cells,Immune cells,Perivascular macrophages,PVM,Perivascular macrophages
977518,15.0,31235.095238,808.761905,4.404762,sagittal3,33,3.526361,42.0,3.761200,33.0,...,Neurons,CNS neurons,"Cholinergic, monoaminergic and peptidergic neu...",Cholinergic and monoaminergic neurons,Neurons,CNS neurons,"Cholinergic, monoaminergic and peptidergic neu...",Cholinergic and monoaminergic neurons,SER_2,"Serotonergic neurons, hindbrain-1"
977519,16.0,31441.180723,854.253012,4.831325,sagittal3,54,4.007333,83.0,4.430817,54.0,...,Neurons,CNS neurons,Spinal cord neurons,NaN,Neurons,CNS neurons,Hindbrain neurons/Spinal cord neurons,Hindbrain neurons/Spinal cord neurons,INH_11,10-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251394,51135.0,12529.654135,30102.127820,5.842105,well3_5,58,4.077537,133.0,4.897840,58.0,...,Glia,CNS glia,Astroependymal cells,Astrocytes,Glia,CNS glia,Astroependymal cells,Astrocytes,AC_6,5-0
251395,51136.0,12392.578512,30191.504132,6.123967,well3_5,53,3.988984,121.0,4.804021,53.0,...,Glia,CNS glia,Astroependymal cells,Astrocytes,Glia,CNS glia,Astroependymal cells,Astrocytes,AC_6,5-0
251396,51137.0,12836.442308,29772.788462,15.788462,well3_5,33,3.526361,52.0,3.970292,33.0,...,Vascular cells,Vascular cells,Vascular cells,Vascular and leptomeningeal cells,Vascular cells,Vascular cells,Vascular cells,Vascular and leptomeningeal cells,VLM_1,Vascular leptomeningeal cells-0
251397,51138.0,13028.637931,29868.379310,14.103448,well3_5,39,3.688879,58.0,4.077538,39.0,...,Vascular cells,Vascular cells,Vascular cells,Vascular smooth muscle cells,Vascular cells,Vascular cells,Vascular cells,Vascular and leptomeningeal cells,VLM_1,Vascular leptomeningeal cells-0


In [9]:
adata_celltype.obs['Tissue_leiden_id'] = list(tissue_region.obs['tissue'])

adata_celltype.obs['Tissue_anatomical_label'] = list(tissue_region.obs['anatomical_region_label'])

adata_celltype.obs['Tissue_anatomical_parent_label'] = list(tissue_region.obs['anatomical_parent_region_label'])

adata_celltype.obs['Tissue_final_label'] = list(tissue_region.obs['revised_tissue'])

In [12]:
hailing_annotation=pd.read_csv('data/20220125_Combined_Final_Manual_Annotation.xlsx - Tissue_region_v3.csv')

In [24]:
for tissue_final_label,tissue_symbol in zip(hailing_annotation['revised_tissue'], hailing_annotation['tissue_region_symbol']):
  adata_celltype.obs.loc[adata_celltype.obs['Tissue_final_label']==tissue_final_label,'Tissue_symbol']=tissue_symbol

In [26]:
for tissue_final_label,tissue_description in zip(hailing_annotation['revised_tissue'], hailing_annotation['Description']):
  adata_celltype.obs.loc[adata_celltype.obs['Tissue_final_label']==tissue_final_label,'Tissue_description']=tissue_description

In [28]:
for tissue_final_label,tissue_color in zip(hailing_annotation['revised_tissue'], hailing_annotation['Color']):
  adata_celltype.obs.loc[adata_celltype.obs['Tissue_final_label']==tissue_final_label,'Tissue_color']=tissue_color

In [31]:
adata_raw=sc.read_h5ad('tissue_region/adata_raw_tissue_id.h5ad')

In [37]:
adata_celltype.raw=adata_raw

In [38]:
adata_celltype.write_h5ad('data/20220313_adata_final.h5ad')